In [31]:
import os
import numpy as np
import argoverse
import shutil
from matplotlib.pyplot import imshow
from argoverse.data_loading.argoverse_tracking_loader import ArgoverseTrackingLoader
from argoverse.utils.ply_loader import load_ply
from argoverse.utils.camera_stats import get_image_dims_for_camera as get_dim
from argoverse.utils.calibration import project_lidar_to_img
from PIL import Image

In [4]:
root_dir = '/home/sean/Desktop/data/'

In [5]:
subroot_dir = root_dir + 'train1/'

In [6]:
argoverse_loader = ArgoverseTrackingLoader(subroot_dir)

In [7]:
print('Total number of logs:',len(argoverse_loader))
argoverse_loader.print_all()
print(argoverse_loader.log_list)

Total number of logs: 3
#images: 474
#lidar: 471
['8a15674a-ae5c-38e2-bc4b-f4156d384072', '5ab2697b-6e3e-3454-a36a-aba2c6f27818', '1d676737-4110-3f7e-bec0-0c90f74c248f']


In [37]:
camL = argoverse_loader.CAMERA_LIST[7] # left stereo
print(camL)
camR = argoverse_loader.CAMERA_LIST[8] # right stereo
(width, height) = get_dim(camL)
print(width)
print(height)
baseline = 0.3

stereo_front_left
2464
2056


In [46]:
for log_id in argoverse_loader.log_list:
    print("processing log:", log_id)
    argoverse_data = argoverse_loader.get(log_id)
    calibL = argoverse_data.get_calibration(camL)
    print(calibL.K[0,0])
    calibR = argoverse_data.get_calibration(camR)
    focalX_px = calibL.K[0,0]
    baseline_focal = focalX_px * baseline
    disparity_dir = subroot_dir + '/' + log_id + '/disparity/'
    if not os.path.isdir(disparity_dir):
        os.makedirs(disparity_dir)
        
    lidar_list = argoverse_data.lidar_list
    lidar_timestamp_list = argoverse_data.lidar_timestamp_list
    lidar_5hz_list = lidar_list[::2] # downsample to sync with stereo camera, same initial time frame
    lidar_timestamp_5hz_list = lidar_timestamp_list[::2]
    
    for idx in range(len(lidar_5hz_list)):
        timestamp = lidar_timestamp_5hz_list[idx]
        print("index: ", idx, "current timestamp: ", timestamp)
        pc = load_ply(lidar_5hz_list[idx])
        
        uv = calibL.project_ego_to_image(pc)
        #print(uv.shape)
        #uv, uv_cam, valid_pts = project_lidar_to_img(pc, calibL, argoverse_loader.CAMERA_LIST[7])
        
        fov_inds = (uv[:, 0] < width - 1) & (uv[:, 0] >= 0) & \
                   (uv[:, 1] < height - 1)& (uv[:, 1] >= 0)
        print(fov_inds.shape)
        num_true = 0
        for pt in fov_inds:
            if(pt == True):
                num_true += 1
        print(num_true)
        fov_inds = fov_inds & (pc[:, 0] > 1) # filters out points that are behind camera
        num_true = 0
        for pt in fov_inds:
            if(pt == True):
                num_true += 1
        print(num_true)
        print(fov_inds.shape)
        
        valid_uv = uv[fov_inds, :]
        print(valid_uv.shape)
        valid_uv = np.round(valid_uv).astype(int)
        
        valid_pc = pc[fov_inds, :]        
        valid_uvd = calibL.project_ego_to_cam(valid_pc)
        
        depth_map = np.zeros((height, width)) - 1
        
        for i in range(valid_uv.shape[0]):
            depth_map[int(valid_uv[i, 1]), int(valid_uv[i, 0])] = valid_uvd[i, 2]

        disp_map = (calibL.K[0,0] * baseline) / depth_map
        print(np.mean(disp_map))
        print(type(disp_map))
        
        #new_im = Image.fromarray(disp_map)
        #imshow(new_im)
        # np.save(disparity_dir + '/' + str(timestamp), disp_map)
        
        break

processing log: 8a15674a-ae5c-38e2-bc4b-f4156d384072
3658.4454222302525
index:  0 current timestamp:  315969684019978000
(93926,)
14431
8423
(93926,)
(8423, 3)
-1095.6295908391448
<class 'numpy.ndarray'>
processing log: 5ab2697b-6e3e-3454-a36a-aba2c6f27818
3640.9264016393313
index:  0 current timestamp:  315972990020197000
(97086,)
13679
8299
(97086,)
(8299, 3)
-1090.4030266836623
<class 'numpy.ndarray'>
processing log: 1d676737-4110-3f7e-bec0-0c90f74c248f
3671.839161483479
index:  0 current timestamp:  315984808019684000
(91487,)
12448
8029
(91487,)
(8029, 3)
-1099.7407501770135
<class 'numpy.ndarray'>
